In [2]:
import tkinter as tk
import random
from tkinter import messagebox

In [3]:
class Game(tk.Tk):
    def __init__(self):
        super().__init__()#由于继承，所以将父类初始化
        self.title("2048")
        #分数
        self.score = 0
        #颜色配置
        self.color_config={
            2:"#%02x%02x%02x"%(255,255,0),
            4:"#%02x%02x%02x"%(255,210,0),
            8:"#%02x%02x%02x"%(28,44,0),
            16:"#%02x%02x%02x"%(28,44,100),
            32:"#%02x%02x%02x"%(28,44,0),
            64:"#%02x%02x%02x"%(28,44,100),
            128:"#%02x%02x%02x"%(28,44,0),
            256:"#%02x%02x%02x"%(28,44,100),
            512:"#%02x%02x%02x"%(28,44,0),
            1024:"#%02x%02x%02x"%(28,44,100),
            2048:"#%02x%02x%02x"%(28,44,0), 
            ' ':"#%02x%02x%02x"%(28,44,151),
        }
        #游戏数字
        self.board_list=[]
        #存放控制
        self.empty_position={}
        #存放所有的按钮
        self.board_button = []
        #创建4*4个按钮
        for i in range(4):
            self.board_button.append([])
            for j in range(4):
                button =tk.Button(self,text='',
                                 font="Times 16 bold",
                                 width=5,
                                 height=2,
                                 fg="white",
                                 bg="#%02x%02x%02x"%(28,44,151))
                self.board_button[i].append(button)
                button.grid(row=i,column=j,padx=3,pady=3)
        #分数标签
        self.score_label = tk.Label(self,text="分数：%s"%self.score)
        self.score_label.grid(row=4,column=0)
        
        #重启按钮
        self.restart_button = tk.Button(self,text="restart")
        #给按钮绑定一个事件
        self.restart_button.bind("<Button-1>",self.init)
        self.restart_button.grid(row=4,column=2)
        
        #绑定按键事件
        self.bind('<Key>',self.move)
        #调用初始化
        self.init(None)
    def init(self,event):
        self.board_list = [
            [' ',' ',' ',' '],
            [' ',' ',' ',' '],
            [' ',' ',' ',' '],
            [' ',' ',' ',' '],
        ]
        self.score = 0
        #清空button的text
        for i in range(4):
            for j in range(4):
                self.board_button[i][j].config(text=self.board_list[i][j],
                                              bg=self.color_config[self.board_list[i][j]])
        #随机生成位置
        while True:
            t1 = (random.randint(0,3),random.randint(0,3))
            t2 = (random.randint(0,3),random.randint(0,3))
            if t1!=t2:
                break
        #生成随机2和4
        self.board_list[t1[0]][t1[1]] = random.randrange(2,5,2)
        self.board_list[t2[0]][t2[1]] = random.randrange(2,5,2)
        #更新控件
        self.score_label.config(text="分数：%s"%self.score)
        self.board_button[t1[0]][t1[1]].config(text=self.board_list[t1[0]][t1[1]],
                                              bg=self.color_config[self.board_list[t1[0]][t1[1]]])
        self.board_button[t2[0]][t2[1]].config(text=self.board_list[t2[0]][t2[1]],
                                               bg=self.color_config[self.board_list[t2[0]][t2[1]]])
        
    def move(self,event):
        flag=True
        print(event.char)
        if event.char == 'w' or event.char == 'W':
            self.move_up()
        elif event.char == 's' or event.char == 'S':
            self.move_down()
        elif event.char == 'a' or event.char == 'A':
            self.move_left()
        elif event.char == 'd' or event.char == 'D':
            self.move_right()
        #是否win
        if self.is_win():
            messagebox.showinfo('恭喜','你赢了')
            print('赢了')
            
        #是否gameover
        if self.is_gameover():
            messagebox.showwarning('恭喜','你输了')
            print('输了')
        #随机添加两个元素
        self.add_board()
        self.update_button()
        
        print(event.char)
    def is_win(self):
        #清空空位
        self.empty_position=[]
        for i in range(4):
            for j in range(4):
                if self.board_list[i][j] == 2048:
                    return True
                elif self.board_list[i][j] == ' ':
                    self.empty_position.append([i,j])
        return False
    def is_gameover(self):
        flag = True
        #没有空位，上下左右没有相邻相等的数字
        #
        if len(self.empty_position)==0:
            #横向
            for l in self.board_list:
                for i in range(len(l)-1):
                    if l[i] == l[i+1]:
                        flag =False
             #纵向
            for l in self.turn_right(self.board_list):
                for i in range(len(l)-1):
                    if l[i] == l[i+1]:
                        flag =False
        else:
            flag=False
        return flag
    def add_board(self):
        temp=[[],[]]
        try:
            for i in range(2):
                temp[i]=self.empty_position.pop()
                self.board_list[temp[i][0]][temp[i][1]]=random.randrange(2,5,2)
        except:
            pass
    def move_up(self):
        self.board_list = self.turn_left(self.board_list)
        self.move_left()
        self.board_list = self.turn_right(self.board_list)      
    def move_down(self):
        self.board_list = self.turn_right(self.board_list)
        self.move_left()
        self.board_list = self.turn_left(self.board_list)
    def move_left(self):
        for i in range(4):
            self.board_list[i] = self.oper_left_row(self.board_list[i])
    def move_right(self):
        for i in range(4):
            #先反向
            self.board_list[i].reverse()
            self.board_list[i] = self.oper_left_row(self.board_list[i])
            self.board_list[i].reverse()
            
    def update_button(self):
        #更新控件
        for i in range(4):
            for j in range(4):
                self.board_button[i][j].config(text=self.board_list[i][j],
                                              bg=self.color_config[self.board_list[i][j]])
        self.score_label.config(text="分数：%s"%self.score)
    #向左处理一行                                                          
    def oper_left_row(self,row):
        #挤到一起
        temp = []
        for item in row:
            if item !=' ':
                temp.append(item)
        #合并
        new_row = []
        flag = True
        for i in range(len(temp)):
            if flag:
                if i+1<len(temp) and temp[i] == temp[i+1]:
                    new_row.append(temp[i]*2)
                    #分数处理
                    self.score += temp[i]*2
                    flag = False
                else :
                    new_row.append(temp[i])
            else:
                flag = True
        n=len(new_row)
        #补齐
        for i in range(len(row)-n):
            new_row.append(' ')
        
        return new_row
    #矩阵右转
    def turn_right(self,matrix):
        return [list(x)[::-1] for x in zip(*matrix)]
    #矩阵左转
    def turn_left(self,matrix):
        a = self.turn_right(matrix)
        b = self.turn_right(a)
        c = self.turn_right(b)
        return c
        

In [ ]:
if __name__=='__main__':
    game = Game()
#     print(game.oper_left_row([2,2,2,' ']))
    game.mainloop()





s
s




a
a
q
q
	
	




W
W
D
D
S
S
D
D
S
S
D
D
S
S
D
D
S
S
W
W
D
D
W
W
D
D
A
A
D
D
A
A
D
D
S
S
D
D
A
A
S
S
D
D
S
S
A
A
D
D
S
S
D
D
W
W
S
S
S
S
S
S
S
S
W
W
S
S
S
S
S
S
S
S
D
D
D
D
D
D
A
A
D
D
A
A
W
W
A
A
W
W
D
D
D
D
A
A
W
W
D
D
W
输了
W
